<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<center>
<h1>Algorísmica Avançada</h1>
<h2>Laboratori 4 - Enumeratius</h2>
</center>
</div>

In [12]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<div class="alert alert-info">
<h2>Avís general per a les Pràctiques de Laboratori</h2>

És molt important <b>comentar tot el vostre codi</b>. Comentar el codi no només facilita a altres persones que el llegeixin la compressió del que esteu fent, sinó també a vosaltres mateixos durant el procés de disseny i implementació. D'aquesta forma podreu entendre el vostre propi codi encara que faci molt temps que no el consulteu. A més, a l'hora de ser corregits, un codi comentat s'entén molt millor, la qual cosa ajuda a traslladar al professor les idees que teníeu a l'intentar implementar-lo i encara més en el cas que no ho hagueu pogut acabar-ho amb èxit. No cal comentar cada línia, però si el que fa cada part.

En segon lloc, és molt important <b>respectar les capçaleres de les funcions i estructura de la funció ```return```</b> que s'indiquin a l'enunciat. Tot i que llegim i corregim els vostres codis en detall, fem servir correctors automàtics per guiar i agilitzar el procés i saber quines solucións funcionen i quines no ja d'entrada. 

Axí doncs, si es demana que la funció tingui la següent estructura: 

```python
def exercici1(param1, param2):
    # implementació de la solució
    return resultat 
```

És important que en el notebook existeixi aquesta funció i que no canvieu el nom dels paràmetres ni el tipus de dada retornada.
    
</div>




<div class="alert alert-success">
    <h1>Problema original: Edificis</h1>
    <p>
        Volem solucionar el puzzle dels edificis. Donada una graella com la que tenim a continuació (esquerra), volem trobar la solució al puzzle (dreta):
        <center><img src="img/sky-intro.png" width='25%'></center>
        Podem pensar que cada una de les caselles és un solar on hi podem posar un edifici de X plantes. En aquest cas un edifici de 1,2,3 o 4 plantes ja que el tauler és de 4x4.<br>
        Els números que trobem a fora de la graella ens indiquen quants edificis són visibles des d'aquesta posició. Un edifici d'alçada 4 tapa qualsevol edifici que hi ha al darrere com podeu veure en la imatge següent:
        <center><img src="img/sky-rule-3-1.png" width='25%'></center>
        A més, en cada fila i columna de la graella només pot haver-hi un sol edifici de cada alçada, és a dir, no es poden repetir.
        <center><img src="img/skyscrapers.png" width='25%'></center>
        L'objectiu és doncs, donada una graella com la inicial, trobar la configuració d'edificis que satisfà les condicions.
    </p>
    
</div>

<div class="alert alert-success">
    <h1>Modificació: Empire State</h1>
    <p>
<b>Modificació:</b> en aquest escenari <b>estem obligats a posar un edifici especial</b>, un gratacels d’alçada 2N (“Empire State”). Aquest edifici es pot situar en qualsevol casella de la graella i compta com el més alt possible a la seva fila i columna.
</p>
    <p>
<b>Restricció de files i columnes:</b> en cada fila i columna, totes les alçades han de ser diferents (incloent 2N si n’hi ha). La resta d’edificis tenen alçades entre 1 i N.
</p>
<p>
    </p>
    
</div>

<div class="alert alert-danger">
    <h1>ATENCIÓ: Verificació de l'heurística</h1>
    <p>
A l'estudi previ he hagut de respondre a la pregunta sobre si l'heurística de posar l'Empire State com a primera peça és una bona heurística per minimitzar el nombre de crides recursives. <b> Comproveu-ho.</b> (Per fer-ho cal verificar-ho)
    </p>
    
</div>

La funció **format_sky** us ajudarà a visualitzar el tauler del problema.

Veieu un exemple de com utilitzar aquesta funció:


In [13]:
import numpy as np
from IPython.display import clear_output
import time
from utils_empire_state import format_sky
# Amb aquesta funció es mostren totes les caselles del tauler
# Les caselles que contenen un zero es mostren buides

# Creem unes restriccions i un tauler 
top, bottom, left, right = [0,0,0], [3,0,0], [0,0,0], [0,0,1]
grid = np.zeros((len(left), len(top)), dtype='int')

print('Tauler:')
print(format_sky(grid, top, bottom, left, right))

Tauler:
              
   +-----------+
   |   |   |   |   
   +-----------+
   |   |   |   |   
   +-----------+
   |   |   |   | 1 
   +-----------+
     3        


La funció **delayed_print** us permet bloquejar temporalment un 'print' per si voleu veure les accions que fa a cada pas el vostre algorisme.<br>
No heu d'utilitzar-la a l'entrega ja que el programa tardarà molt més del necessari, utilitzeu-la només mentre programeu la solució.

In [14]:
def delayed_print(grid, top, bottom, left, right, sleep_time=0.02):
    """
    Funció auxiliar que bloqueja temporalment la funció 'print' per veure l'evolució de l'algorisme.
    """
    print(format_sky(grid, top, bottom, left, right))
    clear_output(wait=True)
    time.sleep(sleep_time)

In [15]:
top, bottom, left, right = [0,0,0], [3,0,0], [0,0,0], [0,0,1]
grid = np.zeros((len(left), len(top)), dtype='int')

for i in range(0, len(left)):
    for j in range(0, len(top)):
        grid[i][j] = np.random.randint(1,4)
        delayed_print(grid, top, bottom, left, right, sleep_time=0.25)

              
   +-----------+
   | 2 | 1 | 2 |   
   +-----------+
   | 1 | 2 | 3 |   
   +-----------+
   | 1 | 2 | 1 | 1 
   +-----------+
     3        


In [28]:
###
# LES VOSTRES FUNCIONS EXTRES AQUÍ
###

def satisfies(grid, x, y, num, top, bottom, left, right):
    """
    Donat un tauler, un punt amb coordenades (x,y) i un nombre 'num', comprova si és possible assignar-lo a la posició (x,y) 
    donades les restriccions del problema.
    
    Params
    ------
    :grid: Una matriu de numpy
    :x,y: Un punt de la matriu on volem posar el nombre 'num'
    :num: Un nombre enter
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.
    
    Returns
    -------
    :b: Un boleà True/False depenent de si el nombre 'num' pot ser col·locat a la casella (x,y) o no.
    """
    
    # EL TEU CODI AQUÍ
    #Mirar que no estigui ni en la colmna ni en la fila
    n = len(grid)
    if num in grid[x]:
        return False

    for i in range(n):
        if(grid[i][y] == num):
            return False
        
        
    grid[x][y] = num
    row = grid[x]

    if left[x] != 0:
        count = 0
        highest = 0
        for val in row:
            if val > highest:
                highest = val
                count += 1
        if count > left[x]:
            grid[x][y] = 0
            return False
    if right[x] != 0:
        count = 0
        highest = 0
        for val in reversed(row):
            if val > highest:
                highest = val
                count += 1
        if count > right[x]:
            grid[x][y] = 0
            return False
    
    # validar columna
    col = [grid[i][y] for i in range(n)]
    if top[y] != 0:
        count = 0
        highest = 0
        for val in col:
            if val > highest:
                highest = val
                count += 1
        if count > top[y]:
            grid[x][y] = 0
            return False
    if bottom[y] != 0:
        count = 0
        highest = 0
        for val in reversed(col):
            if val > highest:
                highest = val
                count += 1
        if count > bottom[y]:
            grid[x][y] = 0
            return False
    
    grid[x][y] = 0
    
    return True
    

def skyscrapper_backtracking(grid, top, bottom, left, right):  
    """
    Funció que implementa l'algorisme de backtracking
    
    Params
    ------
    :grid: Una matriu de tipus numpy
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.
    
    Returns
    -------
    :grid: La matriu plena amb la solució correcta o bé False en cas que no tingui solució.    
    """
    
    # NO esborreu aquestes dues línies, serveien per saber quants cops es crida aquesta funció
    # ----------------------------------------------------------------------------------------
    global num_recursive_calls 
    num_recursive_calls += 1
    # ----------------------------------------------------------------------------------------
    
    # EL TEU CODI AQUÍ
    if(all(cell != 0 for row in grid for cell in row)):
        return grid
    
    if(all(cell == 0 for row in grid for cell in row)):
        for i in range(len(grid)):
            for j in range(len(grid)):
                grid[i][j] = 2*len(grid)
                resultat = skyscrapper_backtracking(grid, top, bottom, left, right)
                if(resultat is not False):
                    return resultat
                grid[i][j]= 0
                
                
    
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            if (grid[i][j] == 0):
                x = i
                y = j

    for n in range(1, len(grid)+ 1):
        if(satisfies(grid, x, y, n, top, bottom, left, right)):
            grid[x][y] = n
            resultat = skyscrapper_backtracking(grid, top, bottom, left, right)
            if (resultat is not False):
                grid = resultat
                return grid
            grid[x][y] = 0

    return False
        

from utils_empire_state import gradable   
#
# NO MODIFIQUEU AQUESTA FUNCIÓ, PODEU MODIFICAR LES ALTRES I/O AFEGIR MÉS
#
@gradable
def skyscrapper(top, bottom, left, right):
    """
    Funció principal del problema. Rep quatre llistes corresponents als nombres que hi ha fora del tauler.
    
    Params
    ------
    :top, bottom, left, right: Els nombres de fora del tauler de la part superior, inferior, esquerra i dreta, respectivament.
    """
    
    # Inicialitzem una matriu de zeros
    grid = np.zeros((len(left), len(top)), dtype='int')

    # Cridem a la funció que soluciona el problema mitjançant backtracking
    sol = skyscrapper_backtracking(grid, top, bottom, left, right)   
    
    return sol

In [29]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([0,0,0],
            [3,0,0],
            [0,0,0],
            [0,0,1])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

              
   +-----------+
   | 6 | 3 | 2 |   
   +-----------+
   | 3 | 2 | 1 |   
   +-----------+
   | 2 | 1 | 3 | 1 
   +-----------+
     3        
Crides recursives: 13
Temps: 0.00s


In [30]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([4,2,1,2,3],
            [1,4,3,2,2],
            [3,2,3,2,1],
            [3,4,1,2,2])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

No solution found


TypeError: 'bool' object is not iterable

In [31]:
num_recursive_calls = 0
t0 = time.perf_counter()
skyscrapper([0,0,3,0],
            [0,2,2,0],
            [0,0,0,0],
            [0,2,0,0])
t1 = time.perf_counter()
print('Crides recursives:', num_recursive_calls)
print(f'Temps: {t1-t0:.2f}s')

             3    
   +---------------+
   | 8 | 2 | 3 | 4 |   
   +---------------+
   | 2 | 1 | 4 | 3 | 2 
   +---------------+
   | 3 | 4 | 1 | 2 |   
   +---------------+
   | 4 | 3 | 2 | 1 |   
   +---------------+
         2   2    
Crides recursives: 17
Temps: 0.00s


<div class="alert alert-warning">

<h3>Expliqueu breument (màxim 5 línies) quina estratègia heu seguit.</h3>
    
<p>LA TEVA RESPOSTA AQUÍ</p>

</div>

<div class="alert alert-warning">
<h1>Entrega de la Prova</h1>
</div>

<div class="alert alert-info">
L'entrega de la prova de pràctica es farà amb el mòdul de <b>ub-grader</b>. Per instal·lar-lo feu servir la següent comanda a terminal:

```
python pip install ub-grader --upgrade
```

O una cel·la del notebook amb:

```
!pip install ub-grader --upgrade
```
<br>

Després, a la cel·la següent canvieu els niubs i noms pels vostres. Executeu-la per generar el vostre report del problema. Finalment, el report es generarà al directori d'aquest notebook amb nom ```report_Membre1_Membre2_ex3.enc```, pugeu aquest fitxer al campus virtual una vegada acabada la prova.

És imprescindible que respecteu el format de les funcions i que el vostre codi estigui degudament comentat.<br><br>

</div>

In [ ]:
from ub_grader import init_students, load_spec, grade

init_students([
    {"niub": "niubMEMBRE1", "nombre": "NOM_MEMBRE1", "apellidos": "COGNOM_MEMBRE1"}, # Canvieu aquests noms i niubs
    {"niub": "niubMEMBRE2", "nombre": "NOM_MEMBRE2", "apellidos": "COGNOM_MEMBRE2"}
])

load_spec(r"file:///./specs/empire_state.json")

result = grade(
    skyscrapper,
    signing_key_path=None
)